<a href="https://colab.research.google.com/github/zodbot/llm_finetuning/blob/main/dpo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import userdata
claude_api_key = userdata.get('CLAUDE_API_KEY')


In [2]:
!pip install transformers>=4.34.0
!pip install trl>=0.7.2
# Install required packages
!pip install anthropic

!pip install torch numpy pandas transformers datasets peft trl accelerate bitsandbytes
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.2/297.2 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 35.0 MB/s eta 0:00:00


In [3]:


# Imports
import json
import time
import re
from typing import Dict, List, Tuple, Any
import random
import torch
from google.colab import userdata

import os
import json
import torch
import numpy as np
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer

In [4]:
# Mount Google Drive for saving/loading data
from google.colab import drive
drive.mount('/content/drive')

BASE_PATH = "/content/drive/MyDrive/resume_skill_scoring"
model_path = f"{BASE_PATH}/fine-tuned-model_Phi-3"
base_model = "microsoft/Phi-3-mini-4k-instruct"


from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import AutoPeftModelForCausalLM
def load_model_method1():
    model = AutoPeftModelForCausalLM.from_pretrained(
        model_path,
        load_in_4bit=True,
        device_map="auto",
        trust_remote_code=True
    )
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    return model, tokenizer

model, tokenizer = load_model_method1()

Mounted at /content/drive


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [5]:
import anthropic

def call_claude_api(prompt: str, api_key: str) -> str:
    """Call real Claude API with current model."""
    try:
        client = anthropic.Anthropic(api_key=api_key)

        message = client.messages.create(
            model="claude-3-5-sonnet-20241022",  # Updated model name
            max_tokens=1000,
            messages=[{"role": "user", "content": prompt}]
        )

        return message.content[0].text

    except Exception as e:
        print(f"Claude API error: {e}")
        return None

# Test Claude connection with new model
test_response = call_claude_api("Hello, can you respond with just 'API working'?", claude_api_key)
print(f"Claude API test: {test_response}")

/tmp/ipython-input-3768300864.py:8: DeprecationWarning: The model 'claude-3-5-sonnet-20241022' is deprecated and will reach end-of-life on October 22, 2025.
Please migrate to a newer model. Visit https://docs.anthropic.com/en/docs/resources/model-deprecations for more information.
  message = client.messages.create(


Claude API test: API working


In [6]:
import json
import time
import re
from typing import Dict, List, Tuple, Any
import random

class DPODataCollector:
    def __init__(self, your_model, your_tokenizer, claude_api_key=None):
        self.your_model = your_model
        self.your_tokenizer = your_tokenizer
        self.collected_pairs = []
        # Note: In practice, you'd use the actual Claude API here
        # For now, we'll simulate Claude responses

    def get_your_model_response(self, resume: str, skill: str) -> Dict[str, Any]:
        """Get response from your fine-tuned model."""

        prompt = f"""<instruction>Evaluate the level of expertise for a specific skill in a resume.</instruction>
<resume>{resume}</resume>
<skill>{skill}</skill>
<rating_scale>
0: Not mentioned
1: Mentioned but no evidence of usage
2: Basic usage demonstrated
3: Moderate competency shown
4: Strong competency with specific achievements
5: Expert level with leadership/teaching in that skill
</rating_scale>
<answer>"""

        inputs = self.your_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to(self.your_model.device)

        with torch.no_grad():
            outputs = self.your_model.generate(
                inputs.input_ids,
                max_new_tokens=200,
                temperature=0.0,
                do_sample=False,
                use_cache=False,
            )

        response = self.your_tokenizer.decode(outputs[0], skip_special_tokens=True)
        generated = response[len(prompt):].strip()

        # Extract components
        score = self.extract_score(generated)
        reasoning = self.extract_reasoning(generated)
        evidence = self.extract_evidence(generated)

        return {
            'score': score,
            'reasoning': reasoning,
            'evidence': evidence,
            'raw_response': generated,
            'prompt': prompt
        }

   # Copy the DPODataCollector class here, but update the get_claude_judge_response method:

    def get_claude_judge_response(self, resume: str, skill: str, your_model_response: Dict) -> Dict[str, Any]:
        """Get Claude's assessment as the judge."""

        judge_prompt = f"""You are evaluating a smaller AI model's resume skill assessment.

          Resume: {resume}
          Skill being evaluated: {skill}
          Small model's response: {your_model_response}

          Please evaluate:
          1. Score Accuracy (1-5): Is the skill level score appropriate?
          2. Evidence Quality (1-5): Are the evidence citations accurate and complete?
          3. Reasoning Quality (1-5): Is the reasoning logical and well-supported?

          Then provide your own assessment of this resume for comparison.

          Respond ONLY with valid JSON in this exact format:
          {{
            "score_accuracy": {{"score": 3, "explanation": "explanation here"}},
            "evidence_quality": {{"score": 4, "explanation": "explanation here"}},
            "reasoning_quality": {{"score": 2, "explanation": "explanation here"}},
            "overall_quality": 3,
            "needs_improvement": true,
            "your_assessment": {{
              "score": 4,
              "reasoning": "your reasoning here",
              "evidence": "your evidence here"
            }}
          }}"""

        # Call real Claude API
        claude_response_text = call_claude_api(judge_prompt, claude_api_key)

        if claude_response_text:
            try:
                # Parse JSON response
                claude_response = json.loads(claude_response_text)
                return claude_response
            except json.JSONDecodeError:
                print(f"Failed to parse Claude response: {claude_response_text}")
                # Fallback to simulation
                return self.simulate_claude_judge(resume, skill, your_model_response)
        else:
            # Fallback to simulation if API fails
            return self.simulate_claude_judge(resume, skill, your_model_response)



    def extract_score(self, text: str) -> int:
        """Extract score from model response."""
        for pattern in [r'<rating>(\d+)', r'<score>(\d+)', r'(\d+)']:
            match = re.search(pattern, text)
            if match:
                try:
                    score = int(match.group(1))
                    if 0 <= score <= 5:
                        return score
                except:
                    continue
        return None

    def extract_reasoning(self, text: str) -> str:
        """Extract reasoning from model response."""
        match = re.search(r'<reasoning>(.*?)</reasoning>', text, re.DOTALL)
        if match:
            return match.group(1).strip()
        return ""

    def extract_evidence(self, text: str) -> str:
        """Extract evidence from model response."""
        match = re.search(r'<evidence>(.*?)</evidence>', text, re.DOTALL)
        if match:
            return match.group(1).strip()
        return ""

    def collect_dpo_pair(self, resume: str, skill: str) -> Dict:
        """Collect one DPO training pair."""

        print(f"Testing: {skill}")

        # Get your model's response
        start_time = time.time()
        your_response = self.get_your_model_response(resume, skill)
        model_time = time.time() - start_time

        # Get Claude's judgment
        judge_response = self.get_claude_judge_response(resume, skill, your_response)

        # Check if this case needs improvement
        if judge_response['needs_improvement']:
            dpo_pair = {
                'prompt': your_response['prompt'],
                'chosen': self.format_response(judge_response['your_assessment']),  # Claude's better response
                'rejected': self.format_response(your_response),  # Your model's response
                'metadata': {
                    'skill': skill,
                    'resume_preview': resume[:200],
                    'your_score': your_response['score'],
                    'claude_score': judge_response['your_assessment']['score'],
                    'score_diff': abs(your_response['score'] - judge_response['your_assessment']['score']),
                    'judgment': judge_response,
                    'model_time': model_time
                }
            }

            self.collected_pairs.append(dpo_pair)
            print(f"  ❌ Collected DPO pair - Your: {your_response['score']}, Claude: {judge_response['your_assessment']['score']}")
            return dpo_pair
        else:
            print(f"  ✅ Good response - Your: {your_response['score']}, Claude: {judge_response['your_assessment']['score']}")
            return None

    def format_response(self, response_dict: Dict) -> str:
        """Format response for DPO training."""
        return f"""<rating>{response_dict['score']}</rating>
<reasoning>{response_dict['reasoning']}</reasoning>
<evidence>{response_dict['evidence']}</evidence>"""

    def batch_collect(self, test_cases: List[Dict], max_pairs: int = 50) -> List[Dict]:
        """Collect DPO pairs from multiple test cases."""

        print(f"🔍 Collecting DPO data from {len(test_cases)} test cases...")
        print(f"Target: {max_pairs} improvement cases")
        print("="*60)

        collected = 0
        total_tested = 0

        for test_case in test_cases:
            if collected >= max_pairs:
                break

            total_tested += 1
            resume = test_case['resume']
            skill = test_case['skill']

            pair = self.collect_dpo_pair(resume, skill)
            if pair:
                collected += 1

            # Progress update
            if total_tested % 10 == 0:
                print(f"\nProgress: {total_tested}/{len(test_cases)} tested, {collected} pairs collected")

        success_rate = collected / total_tested * 100 if total_tested > 0 else 0
        print(f"\n✅ Collection complete!")
        print(f"Tested: {total_tested} cases")
        print(f"Collected: {collected} DPO pairs")
        print(f"Success rate: {success_rate:.1f}% (higher = more issues found)")

        return self.collected_pairs

    def save_dpo_data(self, filepath: str):
        """Save collected DPO pairs to file."""
        with open(filepath, 'w') as f:
            json.dump(self.collected_pairs, f, indent=2)
        print(f"💾 Saved {len(self.collected_pairs)} DPO pairs to {filepath}")

    def analyze_collection(self):
        """Analyze the collected DPO data."""
        if not self.collected_pairs:
            print("No DPO pairs collected yet!")
            return

        print(f"\n📊 DPO DATA ANALYSIS")
        print("="*40)

        # Error types
        score_diffs = [pair['metadata']['score_diff'] for pair in self.collected_pairs]
        avg_score_diff = sum(score_diffs) / len(score_diffs)

        print(f"Total pairs: {len(self.collected_pairs)}")
        print(f"Average score difference: {avg_score_diff:.1f}")
        print(f"Max score difference: {max(score_diffs)}")

        # Skills with most issues
        skill_issues = {}
        for pair in self.collected_pairs:
            skill = pair['metadata']['skill']
            skill_issues[skill] = skill_issues.get(skill, 0) + 1

        print(f"\nSkills with most issues:")
        for skill, count in sorted(skill_issues.items(), key=lambda x: x[1], reverse=True)[:5]:
            print(f"  {skill}: {count} issues")

# Generate synthetic test data
def generate_synthetic_test_cases(num_cases: int = 100) -> List[Dict]:
    """Generate synthetic resumes for testing."""

    skills_to_test = [
        'Python', 'Java', 'JavaScript', 'React', 'Angular',
        'Machine Learning', 'Data Science', 'SQL', 'AWS', 'Docker',
        'Leadership', 'Communication', 'Project Management', 'Teamwork',
        'Problem Solving', 'Research', 'Writing', 'Presentation'
    ]

    test_cases = []

    for i in range(num_cases):
        # Randomly select skill and whether it should be present
        skill = random.choice(skills_to_test)
        skill_present = random.choice([True, False])

        if skill_present:
            # Generate resume WITH the skill
            resume = generate_resume_with_skill(skill)
        else:
            # Generate resume WITHOUT the skill
            resume = generate_resume_without_skill(skill)

        test_cases.append({
            'resume': resume,
            'skill': skill,
            'should_have_skill': skill_present,
            'case_id': f"synthetic_{i}"
        })

    return test_cases

def generate_resume_with_skill(skill: str) -> str:
    """Generate a resume that mentions the skill."""
    # Simple template - in practice, use Claude to generate better ones
    return f"""
John Doe
Software Engineer | john.doe@email.com

EXPERIENCE
Senior Developer at TechCorp (2020-2023)
• Developed applications using {skill}
• Led projects involving {skill} implementation
• Achieved 25% performance improvement using {skill}

SKILLS
• Programming: {skill}, Python, JavaScript
• Frameworks: Various {skill}-based frameworks
"""

def generate_resume_without_skill(skill: str) -> str:
    """Generate a resume that does NOT mention the skill."""
    other_skills = ['Python', 'JavaScript', 'SQL', 'React', 'AWS']
    # Remove the target skill if it's in the list
    other_skills = [s for s in other_skills if s.lower() != skill.lower()]

    return f"""
Jane Smith
Data Analyst | jane.smith@email.com

EXPERIENCE
Data Analyst at DataCorp (2019-2023)
• Analyzed datasets using {random.choice(other_skills)}
• Created visualizations and reports
• Collaborated with cross-functional teams

SKILLS
• Programming: {', '.join(other_skills[:3])}
• Analysis: Excel, Tableau, Statistics
"""

# Usage Example
"""
# Initialize the collector
collector = DPODataCollector(model, tokenizer)

# Generate test cases
test_cases = generate_synthetic_test_cases(50)

# Collect DPO pairs
dpo_pairs = collector.batch_collect(test_cases, max_pairs=25)

# Save the data
collector.save_dpo_data(f"{BASE_PATH}/dpo_training_data.json")

# Analyze what was collected
collector.analyze_collection()
"""

'\n# Initialize the collector\ncollector = DPODataCollector(model, tokenizer)\n\n# Generate test cases\ntest_cases = generate_synthetic_test_cases(50)\n\n# Collect DPO pairs\ndpo_pairs = collector.batch_collect(test_cases, max_pairs=25)\n\n# Save the data\ncollector.save_dpo_data(f"{BASE_PATH}/dpo_training_data.json")\n\n# Analyze what was collected\ncollector.analyze_collection()\n'

In [7]:
# # Initialize the DPO data collector
# print("🚀 Setting up DPO Data Collection Pipeline")
# collector = DPODataCollector(model, tokenizer)

# # Generate synthetic test cases for pipeline testing
# print("📝 Generating synthetic test cases...")
# test_cases = generate_synthetic_test_cases(20)  # Start small for testing

# print(f"Generated {len(test_cases)} test cases")
# print("Sample test case:")
# print(f"  Skill: {test_cases[0]['skill']}")
# print(f"  Should have skill: {test_cases[0]['should_have_skill']}")
# print(f"  Resume preview: {test_cases[0]['resume'][:100]}...")

# # Test the pipeline on a few cases
# print("\n🔬 Testing pipeline with 5 cases...")
# dpo_pairs = collector.batch_collect(test_cases[:5], max_pairs=10)

# # Show what was collected
# print(f"\n📊 Pipeline Results:")
# collector.analyze_collection()

# # Save the data
# if dpo_pairs:
#     collector.save_dpo_data(f"{BASE_PATH}/dpo_test_data_2.json")

#     # Show a sample DPO pair
#     print(f"\n📄 Sample DPO Pair:")
#     sample_pair = dpo_pairs[0]
#     print(f"Skill: {sample_pair['metadata']['skill']}")
#     print(f"Your model score: {sample_pair['metadata']['your_score']}")
#     print(f"Claude score: {sample_pair['metadata']['claude_score']}")
#     print(f"Chosen (Claude): {sample_pair['chosen'][:100]}...")
#     print(f"Rejected (Yours): {sample_pair['rejected'][:100]}...")

In [5]:
# Install TRL for DPO training
!pip install trl

In [6]:
# Update the paths to match your saved data
BASE_PATH = "/content/drive/MyDrive/resume_skill_scoring"
dpo_data_path = f"{BASE_PATH}/dpo_test_data_2.json"  # Your specific file
your_model_path = f"{BASE_PATH}/fine-tuned-model_Phi-3"  # Your current model
dpo_output_dir = f"{BASE_PATH}/dpo-improved-model"  # Where to save improved model

# First, let's check your collected data
print("📊 Checking your DPO data...")
with open(dpo_data_path, 'r') as f:
    dpo_data = json.load(f)

print(f"✅ Found {len(dpo_data)} DPO pairs in {dpo_data_path}")

# # Show what issues were collected
# issues = {}
# for pair in dpo_data:
#     for reason in pair['metadata'].get('collection_reasons', []):
#         issues[reason] = issues.get(reason, 0) + 1

# print(f"📋 Issue breakdown:")
# for issue, count in issues.items():
#     print(f"  {issue}: {count} cases")

# Show a sample to verify format
sample = dpo_data[0]
print(f"\n📄 Sample DPO pair:")
print(f"Skill: {sample['metadata']['skill']}")
print(f"Your score: {sample['metadata']['your_score']} vs Claude: {sample['metadata']['claude_score']}")
# print(f"Collection reasons: {sample['metadata']['collection_reasons']}")

print(f"\n🚀 Ready to start DPO training with {len(dpo_data)} pairs!")

📊 Checking your DPO data...
✅ Found 5 DPO pairs in /content/drive/MyDrive/resume_skill_scoring/dpo_test_data_2.json

📄 Sample DPO pair:
Skill: Docker
Your score: 3 vs Claude: 4

🚀 Ready to start DPO training with 5 pairs!


In [10]:
    # Install/update required packages
    !pip install --upgrade trl datasets

In [10]:
# Import required libraries
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm
import json
import os

# Fix PEFT model to enable training
def fix_peft_model_for_training(model):
    """Fix PEFT model to enable adapter training."""
    print("🔧 FIXING PEFT MODEL FOR TRAINING")
    print("=" * 50)

    print(f"Original training mode: {model.training}")

    # Method 1: Enable training mode and unfreeze adapters
    try:
        print("🔄 Method 1: Setting training mode and enabling adapters...")

        # Set to training mode
        model.train()

        # If the model has enable_adapters method
        if hasattr(model, 'enable_adapters'):
            model.enable_adapters()
            print("✅ Enabled adapters using enable_adapters()")

        # Enable adapter layers specifically
        if hasattr(model, 'enable_adapter_layers'):
            model.enable_adapter_layers()
            print("✅ Enabled adapter layers")

        # Unfreeze LoRA parameters manually
        if hasattr(model, 'base_model') and hasattr(model.base_model, 'model'):
            for name, param in model.named_parameters():
                if 'lora' in name.lower() or 'adapter' in name.lower():
                    param.requires_grad = True
                    print(f"✅ Unfroze: {name}")

        # Check if we now have trainable parameters
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        print(f"📊 Trainable parameters after Method 1: {trainable_params:,}")

        if trainable_params > 0:
            return model, "Method 1 (enable adapters)"

    except Exception as e:
        print(f"⚠️ Method 1 failed: {e}")

    # Method 2: Manually enable adapter training
    try:
        print("\n🔄 Method 2: Manually enabling adapter training...")

        # Set training mode
        model.train()

        # For LoRA models, enable training on adapter weights
        for name, module in model.named_modules():
            if hasattr(module, 'training'):
                module.training = True

            # Enable LoRA adapters
            if 'lora' in name.lower() or 'adapter' in name.lower():
                if hasattr(module, 'set_adapter'):
                    try:
                        module.set_adapter(module.active_adapters)
                    except:
                        pass

                # Unfreeze parameters in this module
                for param in module.parameters():
                    param.requires_grad = True

        # Alternative: unfreeze all parameters with lora/adapter in name
        for name, param in model.named_parameters():
            if any(keyword in name.lower() for keyword in ['lora', 'adapter', 'trainable']):
                param.requires_grad = True
                print(f"✅ Unfroze: {name}")

        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        print(f"📊 Trainable parameters after Method 2: {trainable_params:,}")

        if trainable_params > 0:
            return model, "Method 2 (manual adapter enable)"

    except Exception as e:
        print(f"⚠️ Method 2 failed: {e}")

    # Method 3: Load model in training mode from scratch
    try:
        print("\n🔄 Method 3: Reloading model in training mode...")
        from peft import AutoPeftModelForCausalLM

        # Reload the model with training enabled
        model_path = f"{BASE_PATH}/fine-tuned-model_Phi-3"

        training_model = AutoPeftModelForCausalLM.from_pretrained(
            model_path,
            load_in_4bit=True,
            device_map="auto",
            trust_remote_code=True,
            is_trainable=True  # This is key!
        )

        # Ensure training mode
        training_model.train()

        trainable_params = sum(p.numel() for p in training_model.parameters() if p.requires_grad)
        print(f"📊 Trainable parameters after Method 3: {trainable_params:,}")

        if trainable_params > 0:
            return training_model, "Method 3 (reload with is_trainable=True)"

    except Exception as e:
        print(f"⚠️ Method 3 failed: {e}")

    # Method 4: Force enable all adapter parameters
    try:
        print("\n🔄 Method 4: Force enabling all adapter parameters...")

        model.train()

        # Get all parameter names and enable anything that looks like an adapter
        adapter_keywords = ['lora_A', 'lora_B', 'lora_embedding_A', 'lora_embedding_B',
                           'adapter', 'trainable', 'bias']

        enabled_count = 0
        for name, param in model.named_parameters():
            if any(keyword in name for keyword in adapter_keywords):
                param.requires_grad = True
                enabled_count += 1
                print(f"✅ Enabled: {name} - {param.shape}")

        print(f"📊 Enabled {enabled_count} adapter parameters")

        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        print(f"📊 Total trainable parameters: {trainable_params:,}")

        if trainable_params > 0:
            return model, "Method 4 (force enable adapters)"

    except Exception as e:
        print(f"⚠️ Method 4 failed: {e}")

    print("❌ All methods failed to enable training parameters")
    return None, "All methods failed"

# Alternative: Load a fresh trainable PEFT model
def load_trainable_peft_model():
    """Load a fresh PEFT model that's properly configured for training."""
    print("🔄 Loading fresh trainable PEFT model...")

    try:
        from peft import AutoPeftModelForCausalLM, PeftConfig

        model_path = f"{BASE_PATH}/fine-tuned-model_Phi-3"

        # Load with explicit training configuration
        model = AutoPeftModelForCausalLM.from_pretrained(
            model_path,
            load_in_4bit=True,
            device_map="auto",
            trust_remote_code=True,
            is_trainable=True,
            torch_dtype=torch.float16
        )

        # Force training mode
        model.train()

        # Enable all adapters
        if hasattr(model, 'enable_adapters'):
            model.enable_adapters()

        # Check trainable parameters
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        total_params = sum(p.numel() for p in model.parameters())

        print(f"✅ Fresh model loaded")
        print(f"📊 Trainable: {trainable_params:,} / Total: {total_params:,}")
        print(f"📊 Trainable percentage: {100 * trainable_params / total_params:.2f}%")

        if trainable_params > 0:
            return model
        else:
            print("❌ Fresh model also has no trainable parameters")
            return None

    except Exception as e:
        print(f"❌ Failed to load fresh model: {e}")
        return None

# Working DPO trainer that handles the fixed model
class FixedDPOTrainer:
    """DPO trainer for the fixed PEFT model."""

    def __init__(self, model, tokenizer, learning_rate=1e-4):
        self.model = model
        self.tokenizer = tokenizer
        self.learning_rate = learning_rate

        # Verify the model is trainable
        trainable_params = [p for p in model.parameters() if p.requires_grad]
        if len(trainable_params) == 0:
            raise ValueError("No trainable parameters found!")

        print(f"🎯 FixedDPOTrainer ready with {len(trainable_params)} trainable parameter groups")

    def simple_preference_loss(self, prompt, chosen, rejected):
        """Simple preference learning loss with cache fix."""

        # Tokenize
        chosen_text = prompt + chosen
        rejected_text = prompt + rejected

        chosen_tokens = self.tokenizer(chosen_text, return_tensors="pt", max_length=512, truncation=True, padding=True)
        rejected_tokens = self.tokenizer(rejected_text, return_tensors="pt", max_length=512, truncation=True, padding=True)

        # Move to device
        device = next(self.model.parameters()).device
        chosen_tokens = {k: v.to(device) for k, v in chosen_tokens.items()}
        rejected_tokens = {k: v.to(device) for k, v in rejected_tokens.items()}

        # Forward pass with cache disabled to avoid compatibility issues
        self.model.train()

        try:
            # Try with use_cache=False first
            chosen_outputs = self.model(**chosen_tokens, use_cache=False)
            rejected_outputs = self.model(**rejected_tokens, use_cache=False)
        except AttributeError as e:
            if "get_usable_length" in str(e):
                # Fallback: use minimal parameters
                chosen_outputs = self.model(
                    input_ids=chosen_tokens['input_ids'],
                    attention_mask=chosen_tokens.get('attention_mask', None),
                    use_cache=False,
                    past_key_values=None
                )
                rejected_outputs = self.model(
                    input_ids=rejected_tokens['input_ids'],
                    attention_mask=rejected_tokens.get('attention_mask', None),
                    use_cache=False,
                    past_key_values=None
                )
            else:
                raise e

        # Calculate perplexity-based loss
        chosen_logits = chosen_outputs.logits[:, :-1, :].contiguous()
        rejected_logits = rejected_outputs.logits[:, :-1, :].contiguous()

        chosen_labels = chosen_tokens['input_ids'][:, 1:].contiguous()
        rejected_labels = rejected_tokens['input_ids'][:, 1:].contiguous()

        # Cross entropy losses
        chosen_loss = F.cross_entropy(chosen_logits.view(-1, chosen_logits.size(-1)), chosen_labels.view(-1), reduction='mean')
        rejected_loss = F.cross_entropy(rejected_logits.view(-1, rejected_logits.size(-1)), rejected_labels.view(-1), reduction='mean')

        # Preference loss: chosen should have lower loss than rejected
        margin = 1.0
        preference_loss = torch.clamp(chosen_loss - rejected_loss + margin, min=0.0)

        return preference_loss, chosen_loss, rejected_loss

    def train_dpo(self, dpo_data_path, num_epochs=2):
        """Train using DPO on the fixed model."""

        # Load DPO data
        with open(dpo_data_path, 'r') as f:
            dpo_pairs = json.load(f)

        print(f"📚 Training on {len(dpo_pairs)} DPO pairs")

        # Setup optimizer for trainable parameters only
        trainable_params = [p for p in self.model.parameters() if p.requires_grad]
        optimizer = torch.optim.AdamW(trainable_params, lr=self.learning_rate)

        total_successful = 0

        for epoch in range(num_epochs):
            epoch_loss = 0
            successful_batches = 0

            print(f"\n🔄 Epoch {epoch + 1}/{num_epochs}")

            for i, pair in enumerate(tqdm(dpo_pairs, desc="Training")):
                try:
                    optimizer.zero_grad()

                    # Compute loss
                    pref_loss, chosen_loss, rejected_loss = self.simple_preference_loss(
                        pair['prompt'], pair['chosen'], pair['rejected']
                    )

                    # Backprop
                    pref_loss.backward()

                    # Gradient clipping
                    torch.nn.utils.clip_grad_norm_(trainable_params, max_norm=1.0)

                    # Update
                    optimizer.step()

                    epoch_loss += pref_loss.item()
                    successful_batches += 1

                    # Log every 10 steps
                    if i % 10 == 0:
                        avg_loss = epoch_loss / max(successful_batches, 1)
                        print(f"Step {i}: Pref={avg_loss:.4f}, Chosen={chosen_loss:.4f}, Rejected={rejected_loss:.4f}")

                except Exception as e:
                    print(f"⚠️ Error at step {i}: {e}")
                    continue

            print(f"✅ Epoch {epoch+1}: {successful_batches}/{len(dpo_pairs)} successful")
            total_successful += successful_batches

        print(f"🎉 Training complete! {total_successful} total successful updates")
        return total_successful > 0

# Main execution with fixes
def run_fixed_dpo_training():
    """Main function to fix and train the model."""

    print("🚀 FIXING AND TRAINING PEFT MODEL")
    print("=" * 50)

    global model  # We'll update the global model

    # First try to fix the existing model
    fixed_model, method = fix_peft_model_for_training(model)

    if fixed_model is None:
        print("🔄 Trying to load fresh trainable model...")
        fixed_model = load_trainable_peft_model()
        method = "Fresh model load"

    if fixed_model is None:
        print("❌ Could not create a trainable model!")
        return False

    print(f"✅ Model fixed using: {method}")

    # Update the global model
    model = fixed_model

    # Verify it's trainable
    trainable_count = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"📊 Final trainable parameters: {trainable_count:,}")

    if trainable_count == 0:
        print("❌ Still no trainable parameters!")
        return False

    # Train with DPO
    try:
        trainer = FixedDPOTrainer(model, tokenizer, learning_rate=1e-4)
        dpo_data_path = f"{BASE_PATH}/dpo_test_data_2.json"
        success = trainer.train_dpo(dpo_data_path, num_epochs=2)

        if success:
            print("🎉 DPO training successful!")

            # Save the trained model
            output_dir = f"{BASE_PATH}/fixed-dpo-model"
            model.save_pretrained(output_dir)
            tokenizer.save_pretrained(output_dir)
            print(f"💾 Trained model saved to: {output_dir}")

            return True
        else:
            print("❌ DPO training failed")
            return False

    except Exception as e:
        print(f"❌ Training error: {e}")
        return False

# Run the fix and training
if __name__ == "__main__":
    success = run_fixed_dpo_training()

    if success:
        print("\n🎉 SUCCESS! Your model has been fixed and trained!")
        print("✅ The model now has trainable parameters and completed DPO training")
    else:
        print("\n😞 Failed to fix and train the model")
        print("💡 You may need to reload your PEFT model with is_trainable=True from the start")

🚀 FIXING AND TRAINING PEFT MODEL
🔧 FIXING PEFT MODEL FOR TRAINING
Original training mode: True
🔄 Method 1: Setting training mode and enabling adapters...
⚠️ Method 1 failed: No adapter loaded. Please load an adapter first.

🔄 Method 2: Manually enabling adapter training...
✅ Unfroze: base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight
✅ Unfroze: base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight
✅ Unfroze: base_model.model.model.layers.1.self_attn.o_proj.lora_A.default.weight
✅ Unfroze: base_model.model.model.layers.1.self_attn.o_proj.lora_B.default.weight
✅ Unfroze: base_model.model.model.layers.2.self_attn.o_proj.lora_A.default.weight
✅ Unfroze: base_model.model.model.layers.2.self_attn.o_proj.lora_B.default.weight
✅ Unfroze: base_model.model.model.layers.3.self_attn.o_proj.lora_A.default.weight
✅ Unfroze: base_model.model.model.layers.3.self_attn.o_proj.lora_B.default.weight
✅ Unfroze: base_model.model.model.layers.4.self_attn.o_proj.lora_A.def

Training:  20%|██        | 1/5 [00:02<00:09,  2.42s/it]

Step 0: Pref=1.0638, Chosen=1.5412, Rejected=1.4774


Training: 100%|██████████| 5/5 [00:08<00:00,  1.63s/it]


✅ Epoch 1: 5/5 successful

🔄 Epoch 2/2


Training:  20%|██        | 1/5 [00:01<00:06,  1.55s/it]

Step 0: Pref=0.3478, Chosen=2.0400, Rejected=2.6922


Training: 100%|██████████| 5/5 [00:07<00:00,  1.46s/it]


✅ Epoch 2: 5/5 successful
🎉 Training complete! 10 total successful updates
🎉 DPO training successful!
💾 Trained model saved to: /content/drive/MyDrive/resume_skill_scoring/fixed-dpo-model

🎉 SUCCESS! Your model has been fixed and trained!
✅ The model now has trainable parameters and completed DPO training
